# Mise à disposition du jeu de données stroke_dataset via une API REST

## Description du dataset

Le jeu de données utilisé provient de Kaggle : [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).  
Il contient des données de patients avec différentes caractéristiques médicales et sociales, ainsi que l'information si le patient a subi un accident vasculaire cérébral (AVC) ou non.

Télécharger les données et ajouter les dans un dossier data/.

Les colonnes des données sont :  
- `id` : Identifiant unique du patient  
- `gender` : Sexe  
- `age` : Âge  
- `hypertension` : Présence d'hypertension (0 ou 1)  
- `heart_disease` : Présence de maladie cardiaque (0 ou 1)  
- `ever_married` : Statut marital  
- `work_type` : Type d'emploi  
- `Residence_type` : Urbaine ou rurale  
- `avg_glucose_level` : Moyenne du taux de glucose  
- `bmi` : Indice de masse corporelle  
- `smoking_status` : Statut tabagique  
- `stroke` : Présence d'AVC (0 ou 1)

## Projet

Vous devez exposer les données patients du jeu de données via une API REST afin que les données soit utilisables par d'autres équipes (médecins, data science, étude, etc.).

Cette API REST sera développée avec FastAPI et les spécifications sont les suivantes :
| Méthode | Endpoint                                      | Fonctionnalité                                                                                                    |
| ------- | --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `GET`   | `/patients/{id}`                              | Récupère les détails d’un patient donné (via son identifiant unique)                                              |
| `GET`   | `/patients?stroke=1&gender=Female&max_age=60` | Renvoie les patients filtrés selon plusieurs critères : AVC (oui/non), genre, âge maximal                         |
| `GET`   | `/stats/`                                     | Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes, etc.) |



---

## Quelques définitions


1. Qu’est-ce qu’une API REST ?

- API signifie Application Programming Interface (Interface de Programmation d’Application). C’est un ensemble de règles et de protocoles qui permettent à des logiciels de communiquer entre eux.
- REST signifie Representational State Transfer. C’est un style architectural pour concevoir des services web.
Il en existe d'autres mais REST est celui que vous rencontrerez le plus souvent.
- Vous avez utilisé une API REST via l'API Google Books.

- A quoi sert une API REST ?

    - Permet à différentes applications de communiquer facilement, même si elles sont écrites dans des langages différents.
    - Permet d’accéder à des services distants (ex : bases de données, services web) de manière standardisée.
    - Facilite la création d’applications modulaires et évolutives (front-end, back-end, mobile, etc.)

2. Principes clés d’une API REST

- a. Utilisation du protocole HTTP
Les échanges entre client et serveur utilisent des méthodes HTTP standard comme :

    - GET : pour récupérer des données
    - POST : pour envoyer ou créer des données
    - PUT : pour mettre à jour des données
    - DELETE : pour supprimer des données

- b. Accès aux ressources via des URLs

Chaque ressource (par exemple un livre, un utilisateur) est accessible via une URL unique.

Exemple fictif:
    https://api.example.com/books/123 pour accéder au livre d’identifiant 123.

- c. Stateless (sans état)

Le serveur ne conserve aucune information sur le client entre deux requêtes. Chaque requête doit contenir toutes les informations nécessaires.

- d. Représentations des données

Les données sont envoyées et reçues généralement en format JSON ou XML, qui sont faciles à lire et à manipuler.

- e. Utilisation de codes status HTTP

Chaque réponse du serveur est accompagnée d’un code HTTP indiquant le résultat de la requête. (cf [liste des codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status))

## Outils utilisés
1. FastAPI

FastAPI est un framework Python moderne, rapide et très utilisé dans le milieu professionnel pour construire des API REST. 

Il permet :
- une définition simple des routes et des paramètres  
- La génération automatique de documentation interactive (Swagger UI)  
- FastAPI lit les requêtes entrantes, les traite avec ton code Python, et retourne une réponse HTTP (en JSON).

2. Uvicorn : exécute l'application FastAPI

- Uvicorn est un serveur ASGI (Asynchronous Server Gateway Interface) : c'est une interface standard pour gérer les requêtes de manière asynchrone et performante, notamment utile pour les applications modernes.
- Il attend les requêtes HTTP (par exemple depuis un navigateur), les transmet à FastAPI, et renvoie la réponse.
- Uvicorn permet à l'API de fonctionner : sans Uvicorn ou un autre serveur, FastAPI ne peut pas fonctionner.


3. Swagger UI : l’interface de doc et test interactive

- Swagger UI est généré automatiquement par FastAPI.
- C’est une interface web qui permet de :
    - Voir toutes les routes disponibles dans l'API
    - Tester les routes en envoyant des requêtes sans écrire de code (bouton try it out)
    - Voir les paramètres attendus et les formats de réponse
    
4. Résumé des interactions

- Tester la route de base de l'API grâce à la commande :
```bash
    poetry run fastapi dev stroke_api/main.py
```

--> Qu'est-ce qu'il se passe derrière cette commande ?

- Uvicorn démarre un serveur local
- FastAPI génère automatiquement une interface : Swagger UI, accessible sur http://127.0.0.1:8000/docs qui affiche toutes les routes définies dans le code python FastAPI
- Quand on clique sur "Try it out" dans Swagger UI, Swagger envoie une requête HTTP au serveur (ici Uvicorn)
- Le serveur (Uvicorn) la reçoit, l’envoie à FastAPI, qui traite et renvoie une réponse
- Swagger UI affiche la réponse de l’API (par ex : liste de patients)

---

Import des bibliothèques utiles au projet

In [12]:
import pandas as pd
import numpy as np 
from scipy import stats

## 1. Prétraitement des données / Data preprocessing

Les données réelles sont rarement prêtes à être utilisées directement. Elles peuvent contenir des erreurs, des valeurs manquantes, des doublons, des formats incohérents, ou ne pas être adaptées au modèle ou au système cible.

Le prétraitement consiste à nettoyer, structurer et transformer les données brutes avant de les exploiter dans un projet (modèle IA, API, visualisation, etc.).

Vous avez déjà prétraité des données, petit rappel des éléments sur lesquels travailler dans un prétraitment classique et les méthodes pandas qu'il est possible d'utiliser pour les différentes étapes (des exeples d'utilisation des méthodes pandas sont disponibles dans la doc) : 
- explorer les données pour identifier les types de données, valeurs manquantes, incohérence ([info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html))
- adapter les types si nécessaire ([astypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html))
- identifier les doublons et les supprimer s'il y en a ([duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html), [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html))
- traiter les valeurs manquantes s'il y en a ([fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html))
- identifier les incohérences éventuelles (valeurs aberrantes/outliers) en vérifiant si les valeurs min, max, moyennes sont raisonnables (recherche internet si nécessaire) ([describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)), et les traiter.
- Traiter les valeurs aberrantes si vous en détectez ([loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) pour récupérer les lignes qui répondent à une certaine condition, cf exemple ci-dessous)


**Exemple df.loc :**

Récupérer toutes les lignes de df telles que la valeur de "nom de colonne" >= 0

```df_subset = df.loc[stroke_data_df['nom de colonne'] >= 0]```



---
### **TODO**
1.a. Prétraiter les données du dataset.


1.b. Documenter dans le README.md :
- Les étapes de prétraitement,
- Justification des choix concernant le traitement des valeurs manquantes (si besoin),
- Liste des valeurs raisonnables utilisées pour détecter les valeurs aberrantes, 
- Justification des choix pour traiter les valeurs aberrantes (si besoin).

2.a. Chercher des infos sur le format de fichier parquet et indiquer les sources consultées : 
- Différence principale avec le format csv ?
- Dans quels cas l'utiliser ? 
- Pourquoi c'est un format adapté aux gros volumes de données ?



2.b. Sauvegarder les données prétraiteées dans un fichier parquet ([to_parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)).


In [146]:
# Prétraitement de données
df_health = pd.read_csv("/home/jewao/stroke-api/stroke_api/data/healthcare-dataset-stroke-data.csv")

#j'ai remplis les valuers manquantes par mediane()
df_health["bmi"] = df_health["bmi"].fillna(df_health["bmi"].median())

#j'ai transformé la colonne age en int64, j'affiche l'age >= 1
df_health["age"] = df_health["age"].astype(int)
def category(age):
    if age < 1:
        return 'nourisson'
    elif age < 10:
        return 'enfants'
    elif age < 18:
        return 'adolescents'
    elif age < 60:
        return 'adult'
    else: 
        return 'senior'
print(df_health['age'].apply(category))


#encode in 1 0 format ["ever_married"]
df_health["ever_married"] = df_health["ever_married"].map({'Yes': 1, 'No': 0})

#pour tous ceux qui ont moins de 10 ans , je mets la valeur No dans la colonne ["smoking_status"]
df_health.loc[(df_health["age"] < 10) & (df_health["smoking_status"] == "Unknown"), "smoking_status"] = "No"

#gender to lower case
df_health["gender"] = df_health["gender"].str.lower()

def parquet(df):
    df.to_parquet(path="/home/jewao/stroke-api/fichier_parquet.parquet", index=True)
    print("ok")
# parquet(df_health)

0       senior
1       senior
2       senior
3        adult
4       senior
         ...  
5105    senior
5106    senior
5107     adult
5108     adult
5109     adult
Name: age, Length: 5110, dtype: object


In [147]:
df_health

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,male,67,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,female,61,0,0,1,Self-employed,Rural,202.21,28.1,never smoked,1
2,31112,male,80,0,1,1,Private,Rural,105.92,32.5,never smoked,1
3,60182,female,49,0,0,1,Private,Urban,171.23,34.4,smokes,1
4,1665,female,79,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,female,80,1,0,1,Private,Urban,83.75,28.1,never smoked,0
5106,44873,female,81,0,0,1,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,female,35,0,0,1,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,male,51,0,0,1,Private,Rural,166.29,25.6,formerly smoked,0


-----
## Développement de l'API

A présent que les données sont propres, on peut débuter la création de l'API.

Pour cela, vous allez avoir besoin de quelques fonctions permettant de filtrer les données.

Vous allez les définir ci-dessous, ce qui vous permettra de les tester puis les fonctions seront reportées dans le fichier filters.py.

## Route `/patients/`
- Cette route retourne une liste filtrée de patients
- On souhaite pouvoir filtrer par `gender`, `stroke` ou `max_age`

L'objectif est ici de définir une fonction python qui prend en entrée les paramètres optionnels : _gender_, *stroke*, *max_age* et qui renvoie un dictionnaire filtré des données.

On décompose la rédaction de cette fonction en plusieurs étapes. 

Dans un premier temps, écrire et tester les filtres que l'on souhaite appliquer sur les données (utiliser [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html))

In [ ]:
# Filtrer le dataframe pour ne garder que les patients pour lesquels "stroke=1"
df_health.loc[df_health["stroke"] == 1]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61,0,0,1,Self-employed,Rural,202.21,28.1,never smoked,1
2,31112,Male,80,0,1,1,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49,0,0,1,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
244,17739,Male,57,0,0,1,Private,Rural,84.96,36.7,Unknown,1
245,49669,Female,14,0,0,0,children,Rural,57.93,30.9,Unknown,1
246,27153,Female,75,0,0,1,Self-employed,Rural,78.80,29.3,formerly smoked,1
247,34060,Male,71,1,0,1,Self-employed,Rural,87.80,28.1,Unknown,1


In [141]:
# Filtrer les données pour ne garder que les patients pour lesquels "gender="male"
df_health.loc[df_health["gender"] == "male"]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,male,67,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
2,31112,male,80,0,1,1,Private,Rural,105.92,32.5,never smoked,1
5,56669,male,81,0,0,1,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,male,74,1,1,1,Private,Rural,70.09,27.4,never smoked,1
13,8213,male,78,0,1,1,Private,Urban,219.84,28.1,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5097,64520,male,68,0,0,1,Self-employed,Urban,91.68,40.8,Unknown,0
5098,579,male,9,0,0,0,children,Urban,71.88,17.5,No,0
5099,7293,male,40,0,0,1,Private,Rural,83.94,28.1,smokes,0
5100,68398,male,82,1,0,1,Self-employed,Rural,71.97,28.3,never smoked,0


In [149]:
# Filtrer les données pour ne garder que les patients tels que "age <= max_age"
max_age = df_health["age"].max()
print(max_age)
df_health.loc[df_health["age"] <= max_age]

82


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,male,67,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,female,61,0,0,1,Self-employed,Rural,202.21,28.1,never smoked,1
2,31112,male,80,0,1,1,Private,Rural,105.92,32.5,never smoked,1
3,60182,female,49,0,0,1,Private,Urban,171.23,34.4,smokes,1
4,1665,female,79,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,female,80,1,0,1,Private,Urban,83.75,28.1,never smoked,0
5106,44873,female,81,0,0,1,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,female,35,0,0,1,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,male,51,0,0,1,Private,Rural,166.29,25.6,formerly smoked,0


Appliquer successivement les 3 filtres au sein d'une fonction qui prend en entrée le dataframe, _stroke_, _gender_, _max_age_ et qui renvoie une liste de dictionnaire de patients (utiliser la méthode pandas [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)).

Exemple
```
[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1}]
  ```

In [165]:
def filter_patient(df, max_age, gender, stroke):
    df = df.loc[(df["stroke"] == stroke) & (df["gender"] == gender) & (df["age"] <= max_age)]
    df_dict = df.to_dict(orient='dict')
    for k, v in df_dict.items():
        print(k,v)
filter_patient(df_health, 56, 'male', 1) 

id {31: 33879, 34: 14248, 41: 1261, 79: 42117, 85: 32399, 88: 14499, 94: 54385, 123: 44033, 191: 66071, 204: 62019, 224: 8899}
gender {31: 'male', 34: 'male', 41: 'male', 79: 'male', 85: 'male', 88: 'male', 94: 'male', 123: 'male', 191: 'male', 204: 'male', 224: 'male'}
age {31: 42, 34: 48, 41: 54, 79: 43, 85: 54, 88: 47, 94: 45, 123: 56, 191: 51, 204: 54, 224: 49}
hypertension {31: 0, 34: 0, 41: 0, 79: 0, 85: 0, 88: 0, 94: 0, 123: 1, 191: 1, 204: 0, 224: 0}
heart_disease {31: 0, 34: 0, 41: 0, 79: 0, 85: 0, 88: 0, 94: 0, 123: 0, 191: 0, 204: 0, 224: 0}
ever_married {31: 1, 34: 0, 41: 1, 79: 1, 85: 1, 88: 1, 94: 1, 123: 1, 191: 1, 204: 1, 224: 0}
work_type {31: 'Private', 34: 'Govt_job', 41: 'Private', 79: 'Self-employed', 85: 'Private', 88: 'Private', 94: 'Private', 123: 'Private', 191: 'Private', 204: 'Govt_job', 224: 'Private'}
Residence_type {31: 'Rural', 34: 'Urban', 41: 'Urban', 79: 'Urban', 85: 'Rural', 88: 'Urban', 94: 'Rural', 123: 'Rural', 191: 'Urban', 204: 'Rural', 224: 'Rur

A présent on souhaite ajouter des informations sur les types des paramètres et valeurs de retour de la fonction pour faciliter sa compréhension et son utilisation, ce qu’on appelle l’annotation de type (type hinting).

Cette pratique facilite la lecture et la maintenance du code.

Quels changements pour la fonction ?

A la suite de chaque paramètre, on ajoute le type attendu pour le paramètre. À la suite des paramètres on ajoute le type de ce que qui est retourné par la fonction, dans l'exemple ici : 

```def filter_patient(stroke_data_df: pd.DataFrame, gender: str, etc) -> list[dict]```

Ajouter les types dans la définition de la fonction.

Tester la fonction en ne mettant pas de valeur pour *max_age*.

Que se passe-t-il ?

Dans la fonction écrite ci-dessus, chaque paramètre est obligatoire. 

On souhaite pouvoir filtrer les patients sur 0, 1 ou 2 des paramètres de la fonction (filtrer seulement sur *max_age*  mais ne pas appliquer de filtres sur _gender_ et _stroke_ par exemple).

On peut rendre optionnel les paramètres d'un fonction en choisissant une valeur par défault. Si on utilise la fonction en n'utilisant pas ces paramètres alors la valeur par défault est utilisé.

Copier coller votre fonction ci-dessous et ajouter en paramètre : `max_age=None`

et ajouter la condition suivante **avant le filtre** sur `max_age` : 

```if max_age is not None : ``` 

Si la fonction _filter_patient_ est appelée sans argument *max_age*, alors le filtre sur *max_age* n'est pas appliqué. 

Il est tout à fait possible de définir une valeur par défault par exemple 30 ans : dans ce cas si la fonction est appelée sans argument *max_age*, alors par défault on filtre les patients ayant moins de 30 ans.

**ATTENTION :** Les paramètres optionnels doivent toujours être à la fin de la liste de paramètres.

In [ ]:
# fonction filter_patient paramètre max_age optionnel


In [ ]:
# test fonction sans argument max_age


Ajouter des valeurs par défault et les conditions pour chaque filtre.

Pour les types, on indique qu'il s'agit de paramètres optionels en utilisant le module python _typing_

```
from typing import Optional
def filter_patient(stroke_data_df: pd.DataFrame, gender: Optional[str] = None,etc)
```

Adapter les types en utilisant ce modèle.

In [ ]:
# fonction avec ajout de paramètres par défault et de type

Tester la fonction sans argument pour les filtres, elle doit donc renvoyer le dataframe non filtré.

In [ ]:
# test fonction sans argument pour les filtres

Cette fonction va être utilisée dans la définition de l'API pour créer une route qui permette d'accéder à des données filtrées sur les patients.

Dans le fichier de définition de l'API, toutes les fonctions vont travailler sur les données du fichier. 

Pour alléger les fonctions on va donc utiliser une **variable globale** pour les données et supprimer le paramètre `df` de la fonction.

On lit les données en début de fichier puis on travaille au sein des fonctions sur une copie du dataframe de données.


**En résumé les modifications à faire sont :**


- Supprimer le paramètre df de la fonction,
- Ajouter en début de fonction :  
```df = stroke_data_df.copy()```

1. Dans le fichier filters.py, il suffit d'ajouter : 
- lecture du fichier de données prétraitée dans la variable *df* en début de fichier (utiliser pandas),
- @app.get("/patients/") pour définir le route,
puis la fonction.

2. Dans le fichier api.py: appeler la fonction dans la route correspondante.

Tester la route avec 

```poetry run fastapi dev stroke_api/main.py```

http://127.0.0.1:8000/docs : utiliser la fonctionnalité Try it out pour tester la route.

---
## Autres routes

De la même manière, créer les fonctions appropriées pour la création de :
- la route `/patients/{id}` : Récupère les détails d’un patient donné (via son identifiant unique) 

- la route `/stats/` : Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes).

- Lister les tâches à faire sous forme d'issue github : travailler sur une branche différentes pour l'ajout de chacune des routes.